# imports

In [9]:
import sagemaker

# constants

In [10]:
BUCKET = "classification-by-example-sagemaker"
OUTPUT_PATH = f"s3://{BUCKET}/output"
TEST_PATH = f"s3://{BUCKET}/data/test"
TRAIN_PATH = f"s3://{BUCKET}/data/train"
VALIDATION_PATH = f"s3://{BUCKET}/data/validation"

# set up

In [11]:
sess = sagemaker.Session()
region = sess.boto_region_name
container = sagemaker.image_uris.retrieve(region=region, framework="linear-learner")
role = sagemaker.get_execution_role()
linear = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    instance_type="ml.c4.xlarge",
    output_path=OUTPUT_PATH,
    sagemaker_session=sess,
)
linear.set_hyperparameters(predictor_type="binary_classifier")
linear.set_hyperparameters(mini_batch_size=100)
train_input = sagemaker.inputs.TrainingInput(
    TRAIN_PATH,
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
    record_wrapping=None,
    compression=None,
)
validation_input = sagemaker.inputs.TrainingInput(
    VALIDATION_PATH,
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
    record_wrapping=None,
    compression=None,
)
test_input = sagemaker.inputs.TrainingInput(
    TEST_PATH,
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
    record_wrapping=None,
    compression=None,
)

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: 1.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


# train

In [12]:
%%time
linear.fit(
    inputs={
        "train": train_input,
        "validation": validation_input,
        "test": test_input
    }
)

INFO:sagemaker:Creating training-job with name: linear-learner-2025-03-10-11-53-24-997


2025-03-10 11:53:26 Starting - Starting the training job...
2025-03-10 11:53:40 Starting - Preparing the instances for training...
2025-03-10 11:54:27 Downloading - Downloading the training image............
2025-03-10 11:56:18 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[03/10/2025 11:56:28 INFO 140582999390016] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_bias': '0.0', 'optimizer': 'auto', 'loss': 'auto', 'margin': '1.0', 'quantile': '0.5', 'loss_insen

In [13]:
linear.model_data

's3://classification-by-example-sagemaker/output/linear-learner-2025-03-10-11-53-24-997/output/model.tar.gz'